# Dataprojekt Del 2
Denna notebook kommer innehålla följande steg:
- Data Pre-Processing: 
    - Konvertera relevanta fält till passande data typ
    - Konvertera luftfuktighets data från h till 12 h intervall
    - join data för Nederbördstyp (från nu refererat till som 'typ'), luftfuktighet och lufttemperatur ('temp') till en dataframe, nyckel kommer att vara datum och tid 
- Data Exploration 
    - Plottning av det nya dataframen för att säkerställa att data inte innehåller outliners eller att data fattas
- Data Training
    - Uppdelning av dataset i tränings och validerings data 
    - Test och utvärdering av olika modeller 
- Ytterligare test av model 


Data källa: SMHI's väderstation _Stockholm-Observatoriekullen_.

In [1]:
# Import necessary lib
import pandas as pd


In [7]:
# read CSV files

# TEMP
temp_12h = pd.read_csv('Del2_MetroData/Temp/smhi_lufttemperatur_minMax_12h.csv')
temp_12h_4m = pd.read_csv('Del2_MetroData/Temp/smhi_lufttemperatur_minMax_12h_4m.csv')
# TYP
type_12h  = pd.read_csv('Del2_MetroData/Nederbördstyp/smhi_nederbördstyp_12h.csv')
type_12h_4m  = pd.read_csv('Del2_MetroData/Nederbördstyp/smhi_nederbördstyp_12h_4m.csv')

# Luftfuktighet
humidity_h = pd.read_csv('Del2_MetroData/Luftfuktighet/smhi_relativ_luftfuktighet_h.csv')
humidity_h_4m = pd.read_csv('Del2_MetroData/Luftfuktighet/smhi_relativ_luftfuktighet_h_4m.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Del2_MetroData/Temp/smhi_lufttemperatur_minMax_12h.csv'

## Temperature Analysis

### Temp h
Investigate the dataframe's head

In [132]:
temp_h.head()

,Datum,Tid (UTC),Lufttemperatur,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,1859-01-01,07:00:00,-1.8,G,NaN,Kvalitetskontrollerade historiska data (utom d...
1,1859-01-01,13:00:00,0.9,G,NaN,Tidsperiod (fr.o.m.) = 1859-01-01 00:00:00 (UTC)
2,1859-01-01,20:00:00,1.8,G,NaN,Tidsperiod (t.o.m.) = 2023-01-01 06:00:00 (UTC)
3,1859-01-02,07:00:00,2.0,G,NaN,Samplingstid = Ej angivet
4,1859-01-02,13:00:00,1.8,G,NaN,Kvalitetskoderna:


--> Tidsutsnitt - field not table data 

Investigate shape (rows, columns)

In [133]:
temp_h.shape

(202981, 6)

Check data types
(Objects usually = strings)

In [134]:
temp_h.dtypes

Datum              object
Tid (UTC)          object
Lufttemperatur    float64
Kvalitet           object
Unnamed: 4        float64
Tidsutsnitt:       object
dtype: object

Check out Kvalitet

In [135]:
temp_h.Kvalitet.value_counts()

G    202533
Y       448
Name: Kvalitet, dtype: int64

Från SMHIs dokumentation: 
Grön (G) = Kontrollerade och godkända värden.
Gul (Y) = Misstänkta eller aggregerade värden. Grovt kontrollerade arkivdata och okontrollerade realtidsdata (senaste 2 tim).

--> Övervägande god kvalite. Då SMHI är en svensk expertmyndighet antas deras självuppskattning vara trovärdig. 

### Temp 12h
Investigate the dataframe's head

In [136]:
temp_12h.head()

,Datum,Tid (UTC),Lufttemperatur,Kvalitet,Lufttemperatur.1,Kvalitet.1,Unnamed: 6,Tidsutsnitt:
0,1961-01-01,18:00:00,-0.2,G,1.0,G,NaN,Kvalitetskontrollerade historiska data (utom d...
1,1961-01-02,18:00:00,-0.7,G,1.9,G,NaN,Tidsperiod (fr.o.m.) = 1961-01-01 00:00:00 (UTC)
2,1961-01-03,18:00:00,0.7,G,2.5,G,NaN,Tidsperiod (t.o.m.) = 2023-01-01 06:00:00 (UTC)
3,1961-01-04,18:00:00,0.4,G,2.8,G,NaN,Samplingstid = 12 timmar
4,1961-01-05,18:00:00,-0.3,G,3.0,G,NaN,Kvalitetskoderna:


In [137]:
# Nyaste datapunkten
Newst = temp_12h.Datum.max()

# äldsta datapunkten
Oldest = temp_12h.Datum.min()

print(f'Newst value: {Newst}, oldest value: {Oldest}')


Newst value: 2023-01-01, oldest value: 1961-01-01


In [138]:
temp_12h.shape

(43883, 8)

In [164]:
temp_12h.Kvalitet.value_counts()

G    43557
Y      252
Name: Kvalitet, dtype: int64

## Downfall Analysis

### Downfall mm/h
Investigate the dataframe's head

In [139]:
downfall_mm_h.head()

,Datum,Tid (UTC),Nederbördsmängd,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,NaN,NaN,NaN,NaN,NaN,Kvalitetskontrollerade historiska data (utom d...
1,NaN,NaN,NaN,NaN,NaN,Tidsperiod (fr.o.m.) = 2014-04-01 00:00:00 (UTC)
2,NaN,NaN,NaN,NaN,NaN,Tidsperiod (t.o.m.) = 2014-04-30 23:59:59 (UTC)
3,NaN,NaN,NaN,NaN,NaN,Samplingstid = 1 timme
4,NaN,NaN,NaN,NaN,NaN,Kvalitetskoderna:


In [140]:
downfall_mm_h.dtypes

Datum              float64
Tid (UTC)          float64
Nederbördsmängd    float64
Kvalitet           float64
Unnamed: 4         float64
Tidsutsnitt:        object
dtype: object

In [141]:
downfall_mm_h.shape

(13, 6)

In [142]:
downfall_mm_h.Nederbördsmängd.value_counts()

Series([], Name: Nederbördsmängd, dtype: int64)

Conclusion --> Empty dataset
--> Initiate download and additional analysis of the same dataset type, but from the close by laying station: "Stockholm-Observatoriekullen A" 

In [143]:
downfall_mm_h_A = pd.read_csv('MetroData/smhi_nederbördsmängd_h_kulleA.csv')
downfall_mm_h_A.head()

,Datum,Tid (UTC),Nederbördsmängd,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,1996-10-01,00:00:00,0.0,Y,NaN,Kvalitetskontrollerade historiska data (utom d...
1,1996-10-01,01:00:00,0.0,Y,NaN,Tidsperiod (fr.o.m.) = 1996-10-01 00:00:00 (UTC)
2,1996-10-01,03:00:00,0.0,Y,NaN,Tidsperiod (t.o.m.) = 1997-10-31 23:59:59 (UTC)
3,1996-10-01,04:00:00,0.0,Y,NaN,Samplingstid = 1 timme
4,1996-10-01,05:00:00,0.0,Y,NaN,Kvalitetskoderna:


In [144]:
# Nyaste datapunkten
Newst = downfall_mm_h_A.Datum.max()

# äldsta datapunkten
Oldest = downfall_mm_h_A.Datum.min()

print(f'Newst value: {Newst}, oldest value: {Oldest}')

Newst value: 1997-10-28, oldest value: 1996-10-01


Nyaste datapunk äldre än övriga dataset, antyder att denna station ev. är nerlagd. 

In [145]:
downfall_mm_h_A.Nederbördsmängd.value_counts()

0.0    8731
0.2     112
0.3      90
0.4      48
0.5      34
0.6      27
0.8      20
0.7      19
1.1      15
0.9      12
1.4      10
1.3      10
1.0       9
1.6       8
2.0       8
2.2       7
2.1       5
2.4       5
1.2       4
1.7       4
3.8       4
1.5       3
5.3       3
2.9       3
1.8       3
4.0       3
3.3       3
2.8       2
2.5       2
2.6       2
2.7       2
1.9       2
2.3       2
3.0       2
7.3       1
5.0       1
3.2       1
3.4       1
4.5       1
7.4       1
5.1       1
4.8       1
3.7       1
4.6       1
Name: Nederbördsmängd, dtype: int64

--> Oftast ingen nederbörd uppmätt

In [146]:
downfall_mm_h_A.Kvalitet.value_counts()

Y    9224
Name: Kvalitet, dtype: int64

--> Samtliga mätningar klassade som "Gul (Y) = Misstänkta eller aggregerade värden. Grovt kontrollerade arkivdata och okontrollerade realtidsdata (senaste 2 tim)."
--> detta tillsammans med att perioden inte överlappar med nederbördsTyp data gör detta till ett icke idealt dataset att använda för modellen.

### Downfall Type 12h
Investigate the dataframe's head

In [147]:
downfall_type_12h.head()

,Datum,Tid (UTC),Nederbörd,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,2019-10-07,18:00:00,regn,Y,NaN,Kvalitetskontrollerade historiska data (utom d...
1,2019-10-12,06:00:00,regn,Y,NaN,Tidsperiod (fr.o.m.) = 2019-10-01 00:00:00 (UTC)
2,2019-10-14,06:00:00,regn,Y,NaN,Tidsperiod (t.o.m.) = 2023-01-01 06:00:00 (UTC)
3,2019-10-14,18:00:00,regn,Y,NaN,Samplingstid = 12 timmar
4,2019-10-17,06:00:00,regn,Y,NaN,Kvalitetskoderna:


In [148]:
downfall_type_12h.dtypes

Datum            object
Tid (UTC)        object
Nederbörd        object
Kvalitet         object
Unnamed: 4      float64
Tidsutsnitt:     object
dtype: object

In [159]:
downfall_type_12h.Kvalitet.value_counts()

Y    735
Name: Kvalitet, dtype: int64

--> Samtliga mätningar av sämre kvalitet, kan vara relaterat till att klassning av olika typer av snö/regn inte är numeriskt och på samma sätt inte lika mätbart som t.ex. temperatur eller tid.

In [149]:
downfall_type_12h.Nederbörd.value_counts()

regn                       469
duggregn                    93
snowfall                    77
regnskurar                  46
snöblandat regn             31
snöbyar                     11
snöhagel                     3
byar av snöblandat regn      2
småhagel                     2
kornsnö                      1
Name: Nederbörd, dtype: int64

--> Övervägande lablede data för olika typer av regn 


Inventory of data set size

In [150]:
downfall_type_12h.shape

(735, 6)

In [151]:
# Nyaste datapunkten
Newst = downfall_type_12h.Datum.max()

# äldsta datapunkten
Oldest = downfall_type_12h.Datum.min()

print(f'Newst value: {Newst}, oldest value: {Oldest}')


Newst value: 2022-12-31, oldest value: 2019-10-07


--> Relativt litet dataset jämfört med övriga. Kan kompletteras med data från de senaste tre månaderna, då dessa är vintermånader kan de förhoppningsvis innehålla mer snödata att använda för träningen. 

In [152]:
downfall_type_12h_4months = pd.read_csv('MetroData/smhi_nederbördstyp_12h_last4months.csv')

In [153]:
downfall_type_12h_4months.shape

(100, 6)

In [154]:
downfall_type_12h_4months.Nederbörd.value_counts()

regn               40
snowfall           30
snöblandat regn    16
duggregn            7
snöbyar             3
regnskurar          2
kornsnö             1
snöhagel            1
Name: Nederbörd, dtype: int64

---> Mer jämn fördelning av regn vs snö, stärker den vilda "tesen" att snö är mest förekommande under vintermånader.

In [155]:
# Nyaste datapunkten
Newst = downfall_type_12h_4months.Datum.max()

# äldsta datapunkten
Oldest = downfall_type_12h_4months.Datum.min()

print(f'Newst value: {Newst}, oldest value: {Oldest}')


Newst value: 2023-04-14, oldest value: 2022-12-09


--> Visst överlapp mellan dataseten downfall_type_12h och downfall_type_12h_4months (ca 1 månad), dubbletter behöver tas bort när dataseten senare slås ihop.

In [160]:
downfall_type_12h_4months.Kvalitet.value_counts()

Y    100
Name: Kvalitet, dtype: int64

## Humidity Analysis

### Humidity h
Investigate the dataframe's head

In [156]:
humidity_h.head()

,Datum,Tid (UTC),Relativ Luftfuktighet,Kvalitet,Unnamed: 4,Tidsutsnitt:
0,1961-01-01,06:00:00,93,Y,NaN,Kvalitetskontrollerade historiska data (utom d...
1,1961-01-30,06:00:00,89,Y,NaN,Tidsperiod (fr.o.m.) = 1961-01-01 00:00:00 (UTC)
2,1961-01-31,06:00:00,92,Y,NaN,Tidsperiod (t.o.m.) = 2019-10-31 23:59:59 (UTC)
3,1961-02-01,06:00:00,81,Y,NaN,Samplingstid = Ej angivet
4,1961-02-02,06:00:00,91,Y,NaN,Kvalitetskoderna:


In [157]:
humidity_h.dtypes

Datum                     object
Tid (UTC)                 object
Relativ Luftfuktighet      int64
Kvalitet                  object
Unnamed: 4               float64
Tidsutsnitt:              object
dtype: object

In [162]:
humidity_h.value_counts('Relativ Luftfuktighet')

Relativ Luftfuktighet
92    234
90    193
91    188
86    181
89    174
     ... 
21      1
20      1
19      1
16      1
14      1
Length: 84, dtype: int64

In [163]:
humidity_h.Kvalitet.value_counts()

G    5684
Y    1691
Name: Kvalitet, dtype: int64